In [5]:
!pip install -U spacy --q
!python -m spacy download ru_core_news_lg --q
!pip install pymorphy3 --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 55.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 102.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependenc

In [6]:
import spacy
from spacy.tokens import Span
from spacy import displacy

from pymorphy3 import MorphAnalyzer

import types
from types import NoneType

In [26]:
text = '''Мама мыла оконную раму. Из неё выпало стекло. Оно разбилось о пол.
На полу спал наш пёс шарик. Он услышал звук бьющегося стекла. Шарик залаял на маму.
Мама включила телевизор. На Первом Канале выступал Григорий Лепс. Недавно Правительство РФ присвоило ему звание Народного Артиста РФ.'''

In [27]:
nlp = spacy.load("ru_core_news_lg")
doc = nlp(text)

morph = MorphAnalyzer()

In [8]:
def head_in_named_entity(doc, span): # на вход подаются документ и именованная сущность, которая в нём содержится
    span_parts = span.text.split(' ')
    head = None
    heads = [[], []] # [[token], [head]]
    for span_part in span_parts:
        for token in doc: # перебираем токены, потому что именно они, в отличие от строк, содержат всю информацию
            if span_part == token.text:
                heads[0].append(token)
                heads[1].append(token.head)
    for i in range(len(span_parts)):
        if heads[1][i] not in heads[0]: # вершиной является то, что не зависит от других слов, входящих в именованную сущность
            head = heads[0][i]
    return head, [_.split('=')[1] for _ in str(head.morph).split('|')], head.head, head.dep_, heads

In [9]:
def normalize_noun_phrase(doc, np): # на вход подаётся документ и именная группа
    head, morphology, parent, dep, np_parts = head_in_named_entity(doc, np)
    ana = morph.parse(head.text)[0]
    res = ''
    for i, np_part in enumerate(np_parts[0]):
        np_part_head = np_parts[1][i]
        if np_part == head:
            np_part = ana.normal_form
        else:
            np_part = morph.parse(np_part.text)[0]
            pos = str(np_part.tag).split(',')[0].split(' ')[0]
            if pos == 'ADJF' and np_part_head == head:
                gender, number = str(ana.normalized.tag).split(',')[2].split()
                np_part = np_part.inflect({gender, 'nomn'})[0]
            else:
                np_part = np_part.word
        res += np_part + ' '
    return res.strip()

In [29]:
def get_syntactic_relations(doc):
    chunks = [] # [((индекс первого символа, индекс последнего символа), чанк в тексте, нормализованный чанк, морфологические признаки чанка, родитель чанка, тип зависимости}
    res = [] # [(Концепция1, глагол, Концепция2)]
    subs_and_preds = {} # {сказуемое: подлежащее}
    for ent in doc.ents: # добавляем именованные сущности
        chars = (ent.start_char, ent.end_char)
        chunks.append((chars, ent, normalize_noun_phrase(doc, ent)) + head_in_named_entity(doc, ent)[1:-1])
    for token in doc: # добавляем существительные
        if token.pos_ == 'NOUN':
            morph = [_.split('=')[1] for _ in str(token.morph).split('|')]
            chars = (token.idx, token.idx + len(token.text))
            chunks.append((chars, token, token.lemma_, morph, token.head, token.dep_))
    chunks.sort(key=lambda x: x[0])
    for token in doc: # решаем анафору
        if token.pos_ == 'PRON':
            morph = [_.split('=')[1] for _ in str(token.morph).split('|')]
            for chunk in chunks:
                if chunk[0][0] < token.idx and chunk[3][2:4] == morph[1:3]:
                    chars = (token.idx, token.idx + len(token.text))
                    pron_chunk = (chars, token, normalize_noun_phrase(doc, chunk[1]), morph, token.head, token.dep_)
            chunks.append(pron_chunk)
    for chunk in chunks:
        if chunk[5] == 'nsubj':
            subs_and_preds[chunk[4]] = chunk[2]
    for chunk in chunks:
        if (type(chunk[1]) == Span or chunk[1].pos_ == 'NOUN' or chunk[1].pos_ == 'PRON') and chunk[4].pos_ == 'VERB' and chunk[5] != 'nsubj':
            res.append((subs_and_preds[chunk[4]], chunk[4].text, chunk[2], chunk[5]))
    return res

In [28]:
get_syntactic_relations(doc)

[((0, 4), Мама, 'мама', ['Anim', 'Nom', 'Fem', 'Sing'], мыла, 'nsubj'), ((18, 22), раму, 'рама', ['Inan', 'Acc', 'Fem', 'Sing'], мыла, 'obj'), ((38, 44), стекло, 'стекло', ['Inan', 'Nom', 'Neut', 'Sing'], выпало, 'nsubj'), ((62, 65), пол, 'пол', ['Inan', 'Acc', 'Masc', 'Sing'], разбилось, 'obl'), ((70, 74), полу, 'пол', ['Inan', 'Loc', 'Masc', 'Sing'], спал, 'obl'), ((88, 93), шарик, 'шарик', ['Inan', 'Nom', 'Masc', 'Sing'], спал, 'nsubj'), ((106, 110), звук, 'звук', ['Inan', 'Acc', 'Masc', 'Sing'], услышал, 'obj'), ((121, 127), стекла, 'стекло', ['Inan', 'Gen', 'Neut', 'Sing'], звук, 'nmod'), ((129, 134), Шарик, 'шарик', ['Anim', 'Nom', 'Masc', 'Sing'], залаял, 'nsubj'), ((145, 149), маму, 'мама', ['Anim', 'Acc', 'Fem', 'Sing'], залаял, 'obl'), ((151, 155), Мама, 'мама', ['Anim', 'Nom', 'Fem', 'Sing'], включила, 'nsubj'), ((165, 174), телевизор, 'телевизор', ['Inan', 'Acc', 'Masc', 'Sing'], включила, 'obj'), ((179, 192), Первом Канале, 'первый канал', ['Inan', 'Loc', 'Masc', 'Sing'], 

[('мама', 'мыла', 'рама', 'obj'),
 ('стекло', 'разбилось', 'пол', 'obl'),
 ('шарик', 'спал', 'пол', 'obl'),
 ('шарик', 'услышал', 'звук', 'obj'),
 ('шарик', 'залаял', 'мама', 'obl'),
 ('мама', 'включила', 'телевизор', 'obj'),
 ('григорий лепс', 'выступал', 'первый канал', 'obl'),
 ('правительство', 'присвоило', 'звание', 'obj'),
 ('стекло', 'выпало', 'рама', 'obl'),
 ('правительство', 'присвоило', 'григорий лепс', 'iobj')]